# Quantum Simulation of Bell States with Noise Models and Interactive Visualization

This notebook simulates a two-qubit quantum system, primarily a Bell state, under various noise models like phase noise, amplitude damping, depolarizing noise, bit flip, and phase flip. It computes probabilities, fidelity, and entanglement, visualizes results with plots and Bloch spheres, and allows interactive exploration via widgets.

By: **Akhilesh Pant** (MCA)

**Amrapali University**, Haldwani

In [1]:
# ===============================================
# Quantum Simulation – Fully Interactive Notebook
# ===============================================

# -------------------------------
# 1. Import Libraries
# -------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qutip import (basis, tensor, ket2dm, fidelity, concurrence,
                   destroy, sigmax, sigmaz, qeye, mesolve, Bloch)
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------------------------------
# 2. Qubit Definitions
# -------------------------------
def create_single_qubit(state="plus"):
    if state == "zero":
        return basis(2,0)
    elif state == "one":
        return basis(2,1)
    elif state == "plus":
        return (basis(2,0) + basis(2,1)).unit()
    elif state == "minus":
        return (basis(2,0) - basis(2,1)).unit()

def create_bell_pair():
    q0 = basis(2,0)
    q1 = basis(2,1)
    return (tensor(q0,q0) + tensor(q1,q1)).unit()

# -------------------------------
# 3. Noise Models
# -------------------------------
def apply_phase_noise(rho, gamma=0.1, tlist=[0,1]):
    sz1 = tensor(sigmaz(), qeye(2))
    sz2 = tensor(qeye(2), sigmaz())
    L = [gamma*sz1, gamma*sz2]
    result = mesolve(H=0*rho, rho0=rho, tlist=tlist, c_ops=L)
    return result.states[-1]

def apply_amplitude_damping(rho, gamma=0.1, tlist=[0,1]):
    c_ops = [gamma*tensor(destroy(2), qeye(2)), gamma*tensor(qeye(2), destroy(2))]
    result = mesolve(H=0*rho, rho0=rho, tlist=tlist, c_ops=c_ops)
    return result.states[-1]

def apply_depolarizing_noise(rho, p=0.1):
    I = qeye(2)
    return (1-p)*rho + (p/4)*tensor(I,I)

def apply_bit_flip(rho, p=0.1):
    X = sigmax()
    return (1-p)*rho + p*tensor(X,X)*rho*tensor(X,X)

def apply_phase_flip(rho, p=0.1):
    Z = sigmaz()
    return (1-p)*rho + p*tensor(Z,Z)*rho*tensor(Z,Z)

# -------------------------------
# 4. Measurement Functions
# -------------------------------
def compute_probabilities(rho):
    rho_q0 = rho.ptrace(0)
    rho_q1 = rho.ptrace(1)
    return {
        "q0_0": (rho_q0*rho_q0.dag()).diag()[0].real,
        "q0_1": (rho_q0*rho_q0.dag()).diag()[1].real,
        "q1_0": (rho_q1*rho_q1.dag()).diag()[0].real,
        "q1_1": (rho_q1*rho_q1.dag()).diag()[1].real
    }

def compute_fidelity(rho, rho_target):
    return fidelity(rho, rho_target)

def compute_entanglement(rho):
    return concurrence(rho)

# -------------------------------
# 5. Simulation Function
# -------------------------------
def run_simulation(noise_type, gamma, p, t_steps, runs):
    bell_state = create_bell_pair()
    bell_density = ket2dm(bell_state)
    tlist = np.linspace(0, t_steps, max(t_steps, 2))
    
    prob_results, fidelity_results, ent_results = [], [], []
    
    for _ in range(runs):
        if noise_type == "Phase Noise":
            rho = apply_phase_noise(bell_density, gamma=gamma, tlist=tlist)
        elif noise_type == "Amplitude Damping":
            rho = apply_amplitude_damping(bell_density, gamma=gamma, tlist=tlist)
        elif noise_type == "Depolarizing Noise":
            rho = apply_depolarizing_noise(bell_density, p=p)
        elif noise_type == "Bit Flip":
            rho = apply_bit_flip(bell_density, p=p)
        elif noise_type == "Phase Flip":
            rho = apply_phase_flip(bell_density, p=p)
        else:
            rho = bell_density
        
        prob_results.append(compute_probabilities(rho))
        fidelity_results.append(compute_fidelity(rho, bell_density))
        ent_results.append(compute_entanglement(rho))
    
    avg_prob = {k: np.mean([r[k] for r in prob_results]) for k in prob_results[0]}
    avg_fidelity = np.mean(fidelity_results)
    avg_ent = np.mean(ent_results)
    
    return avg_prob, avg_fidelity, avg_ent

# -------------------------------
# 6. Plot Functions
# -------------------------------
def plot_probabilities(prob_dict):
    plt.figure(figsize=(6,4))
    plt.bar(prob_dict.keys(), prob_dict.values(), color='skyblue')
    plt.ylabel("Probability")
    plt.title("Qubit Probabilities")
    plt.show()

def plot_entanglement(ent):
    plt.figure(figsize=(4,3))
    plt.bar(["Entanglement"], [ent], color='orange')
    plt.ylabel("Concurrence")
    plt.title("Entanglement")
    plt.show()

def plot_fidelity(f):
    plt.figure(figsize=(4,3))
    plt.bar(["Fidelity"], [f], color='green')
    plt.ylabel("Fidelity")
    plt.title("Fidelity vs Original Bell State")
    plt.show()

def show_bloch():
    b = Bloch()
    single_qubit = create_single_qubit("plus")
    b.add_states([single_qubit])
    b.render()  # <- Works inline in Jupyter

# -------------------------------
# 7. Widgets Interface
# -------------------------------
noise_dropdown = widgets.Dropdown(
    options=['None', 'Phase Noise', 'Amplitude Damping', 'Depolarizing Noise', 'Bit Flip', 'Phase Flip'],
    value='Phase Noise',
    description='Noise Type:',
)

gamma_slider = widgets.FloatSlider(
    value=0.05,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Gamma:',
)

p_slider = widgets.FloatSlider(
    value=0.05,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Probability p:',
)

time_steps_slider = widgets.IntSlider(
    value=50,
    min=2,
    max=200,
    step=1,
    description='Time Steps:',
)

runs_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='Number of Runs:',
)

run_button = widgets.Button(description="Run Simulation", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        noise_type = noise_dropdown.value
        gamma = gamma_slider.value
        p = p_slider.value
        t_steps = time_steps_slider.value
        runs = runs_slider.value
        
        avg_prob, avg_fidelity, avg_ent = run_simulation(noise_type, gamma, p, t_steps, runs)
        
        print(f"Simulation Results for {noise_type}:")
        print(f"Average Probabilities: {avg_prob}")
        print(f"Average Fidelity: {avg_fidelity:.4f}")
        print(f"Average Entanglement: {avg_ent:.4f}")
        
        plot_probabilities(avg_prob)
        plot_fidelity(avg_fidelity)
        plot_entanglement(avg_ent)
        show_bloch()

run_button.on_click(on_button_clicked)

display(noise_dropdown, gamma_slider, p_slider, time_steps_slider, runs_slider, run_button, output)


Dropdown(description='Noise Type:', index=1, options=('None', 'Phase Noise', 'Amplitude Damping', 'Depolarizin…

FloatSlider(value=0.05, description='Gamma:', max=1.0, step=0.01)

FloatSlider(value=0.05, description='Probability p:', max=1.0, step=0.01)

IntSlider(value=50, description='Time Steps:', max=200, min=2)

IntSlider(value=1, description='Number of Runs:', max=20, min=1)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()